## Graph Data Structure And Algorithms
**Source**:https://www.geeksforgeeks.org/graph-data-structure-and-algorithms/

In [11]:
# general graph class (handy!)
from collections import defaultdict, deque


class Graph(object):
    def __init__(self,vertices=list()):
        self.vertices = vertices
        self.edges = defaultdict(list)

    def add_vertex(self,v):
        self.vertices.append(v)

    def add_edge(self,s,t):
        self.edges[s].append(t)

    def add_edges(self,edge_list,type="directed"):
        for pair in edge_list:
            s,t = pair
            self.edges[s].append(t)
            if type=="undirected":
                self.edges[t].append(t)

    def add_weighted_edge(self,s,t,w):
        self.edges[s].append((t,w))

    def add_weighted_edges(self,edge_list,type="directed"):
        for triple in edge_list:
            s,t,w = triple
            self.edges[s].append((t,w))
            if type=="undirected":
                self.edges[t].append((s,w))


## Connectivity

## Topological Sorting

In [4]:
def dfs(g,node,visited,sol):
    visited.add(node)
    for neigh in g.edges[node]:
        if neigh not in visited:
            dfs(g,neigh,visited,sol)
    sol.append(node)


# Topological sorting (only for DAGs)
def topological_sort(g):
    visited = set()
    sol = deque()
    for node in g.vertices:
        if node not in visited:
            dfs(g,node,visited,sol)
    return [x for x in reversed(sol)]

g = Graph(list(range(6)))
g.add_edges([(2,3),(3,1),(4,0),(4,1),(5,0),(5,2)])
print(g.edges)
print(topological_sort(g))

defaultdict(<class 'list'>, {2: [3], 3: [1], 4: [0, 1], 5: [0, 2]})
[5, 4, 2, 3, 1, 0]


## Shortest and Longest Path

### Shortest path in a DAG with O(V + E)
For a general weighted graph, we can calculate single source shortest distances in O(VE) time using Bellman–Ford Algorithm. For a graph with no negative weights, we can do better and calculate single source shortest distances in O(E + VLogV) time using Dijkstra’s algorithm. Can we do even better for Directed Acyclic Graph (DAG)? We can calculate single source shortest distances in O(V+E) time for DAGs

In [5]:
# https://www.geeksforgeeks.org/shortest-path-for-directed-acyclic-graphs/

def dfs(g,node,visited,sol):
    visited.add(node)
    for neigh,_ in g.edges[node]:
        if neigh not in visited:
            dfs(g,neigh,visited,sol)
    sol.append(node)

# Topological sorting (only for DAGs)
def topological_sort(g):
    visited = set()
    sol = deque()
    for node in g.vertices:
        if node not in visited:
            dfs(g,node,visited,sol)
    return [x for x in reversed(sol)]

def find_shortest_path_in_DAG(g,source):
    sorted_vs = topological_sort(g)
    print(sorted_vs)
#     dist = defaultdict(float("inf"))
    dist = defaultdict(lambda:float("inf"))
    dist[source]=0
    # u-->v
    for u in sorted_vs:
        for v,w in g.edges[u]:
            if dist[v] > dist[u] + w:
                dist[v] = dist[u] + w
    return dist
    

# create a weighted DAG
g = Graph(list(range(6)))
g.add_weighted_edges([(0,1,5),(0,2,3),(1,3,6),(1,2,2),(2,4,4),
                     (2,5,2),(2,3,7),(3,4,-1),(4,5,-2)])

print(g.edges)
print(find_shortest_path_in_DAG(g,4))



defaultdict(<class 'list'>, {0: [(1, 5), (2, 3)], 1: [(3, 6), (2, 2)], 2: [(4, 4), (5, 2), (3, 7)], 3: [(4, -1)], 4: [(5, -2)]})
[0, 1, 2, 3, 4, 5]
defaultdict(<function find_shortest_path_in_DAG.<locals>.<lambda> at 0x7f0004622268>, {0: inf, 1: inf, 2: inf, 3: inf, 4: 0, 5: -2})


### Longest Path in DAG
The longest path problem for a general graph is not as easy as the shortest path problem because the longest path problem doesn’t have optimal substructure property. In fact, the Longest Path problem is NP-Hard for a general graph. However, the longest path problem has a linear time solution for directed acyclic graphs. The idea is similar to linear time solution for shortest path in a directed acyclic graph., we use Topological Sorting.

In [31]:
# https://www.geeksforgeeks.org/find-longest-path-directed-acyclic-graph/
def find_longest_path_in_DAG(g,source):
    sorted_vs = topological_sort(g)
    print(sorted_vs)
#     dist = defaultdict(float("inf"))
    dist = defaultdict(lambda:-float("inf"))
    dist[source]=0
    # u-->v
    for u in sorted_vs:
        for v,w in g.edges[u]:
            if dist[v] < dist[u] + w:
                dist[v] = dist[u] + w
    return dist
    

# create a weighted DAG
g = Graph(list(range(6)))
g.add_weighted_edges([(0,1,5),(0,2,3),(1,3,6),(1,2,2),(2,4,4),
                     (2,5,2),(2,3,7),(3,4,-1),(4,5,-2)])

print(g.edges)
print(find_longest_path_in_DAG(g,1))

defaultdict(<class 'list'>, {0: [(1, 5), (2, 3)], 1: [(3, 6), (2, 2)], 2: [(4, 4), (5, 2), (3, 7)], 3: [(4, -1)], 4: [(5, -2)]})
[0, 1, 2, 3, 4, 5]
defaultdict(<function find_longest_path_in_DAG.<locals>.<lambda> at 0x7f000460aa60>, {0: -inf, 1: 0, 2: 2, 3: 9, 4: 8, 5: 6})


### Shortest path to target in DAG

In [8]:
def find_shortest_path_to_target_in_DAG(g,target):
    sorted_vs = topological_sort(g)
    print(sorted_vs)
#     dist = defaultdict(float("inf"))
    dist = defaultdict(lambda:float("inf"))
    dist[target]=0
    # u-->v
    for v in sorted_vs[::-1]:
        for u,w in g.edges[v]:
            if dist[u] > dist[v] + w:
                dist[u] = dist[v] + w
    return dist
    

# create a weighted DAG
g = Graph(list(range(6)))
g.add_weighted_edges([(0,1,5),(0,2,3),(1,3,6),(1,2,2),(2,4,4),
                     (2,5,2),(2,3,7),(3,4,-1),(4,5,-2)])

print(g.edges)
print(find_shortest_path_to_target_in_DAG(g,2))

defaultdict(<class 'list'>, {0: [(1, 5), (2, 3)], 1: [(3, 6), (2, 2)], 2: [(4, 4), (5, 2), (3, 7)], 3: [(4, -1)], 4: [(5, -2)]})
[0, 1, 2, 3, 4, 5]
defaultdict(<function find_shortest_path_to_target_in_DAG.<locals>.<lambda> at 0x7f0004622400>, {0: inf, 1: inf, 2: 0, 3: 7, 4: 4, 5: 2})


### Djikstra!

In [35]:
import heapq

def find_shortest_path(source,target,prev_nodes):
    path = str(target)
    print(prev_nodes)
    node = target
    while prev_nodes[node]!=None:
        print(node)
        path += str(prev_nodes[node])
        node = prev_nodes[node]
    return path[::-1]


def Dijkstra(g,source):
    dist_dict = defaultdict(lambda:float("inf"))
    dist_dict[source] = 0
    prev_node = defaultdict(lambda:None)
    visited = set()
    priority_queue = [(0,source)]
    heapq.heapify(priority_queue)
    while priority_queue:
        cur_node = heapq.heappop(priority_queue)[1]
        if cur_node in visited:
            continue
        for neigh,weight in g.edges[cur_node]:
            dist = dist_dict[cur_node] + weight
            if dist < dist_dict[neigh]:
                dist_dict[neigh] = dist
                prev_node[neigh] = cur_node
                heapq.heappush(priority_queue,(dist,neigh))

        visited.add(cur_node)
    return dist_dict,prev_node

g = Graph(list(range(9)))
g.add_weighted_edges([(0,1,4),(0,7,8),(1,2,8),(1,7,11),(2,3,7),(2,8,2),(2,5,4),(3,4,9),
                     (3,5,14),(4,5,10),(5,6,2),(6,7,1),(6,8,6),(7,8,7)],type="undirected")
print(g.edges)
dist,prevs = Dijkstra(g,0)
print(dist)
print(find_shortest_path(0,4,prevs))



defaultdict(<class 'list'>, {0: [(1, 4), (7, 8)], 1: [(0, 4), (2, 8), (7, 11)], 2: [(1, 8), (3, 7), (8, 2), (5, 4)], 3: [(2, 7), (4, 9), (5, 14)], 4: [(3, 9), (5, 10)], 5: [(2, 4), (3, 14), (4, 10), (6, 2)], 6: [(5, 2), (7, 1), (8, 6)], 7: [(0, 8), (1, 11), (6, 1), (8, 7)], 8: [(2, 2), (6, 6), (7, 7)]})
defaultdict(<function Dijkstra.<locals>.<lambda> at 0x7f0004598378>, {0: 0, 1: 4, 2: 12, 3: 19, 4: 21, 5: 11, 6: 9, 7: 8, 8: 14})
defaultdict(<function Dijkstra.<locals>.<lambda> at 0x7f00045982f0>, {1: 0, 2: 1, 3: 2, 4: 5, 5: 6, 6: 7, 7: 0, 8: 2})
4
5
6
7
07654
